In [ ]:
# cd ~/twinews-logs ; jupython --no-nn --venv st-venv ~/notebooks/twinews/annot-multinews.ipynb

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
from annotator.annot import *
from systemtools.location import *
from systemtools.hayj import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *

In [ ]:
import sys ; sys.path.append(homeDir() + '/Workspace/Python/Datasets/Twinews')
from twinews.utils import *

In [ ]:
logger = Logger(tmpDir('logs') + "/annot-multinews.log") if isNotebook else Logger("annot-multinews.log")

In [ ]:
(hjuser, hjpass, hjhost) = getMongoAuth(user='hayj')

In [ ]:
newsCollection = getNewsCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)
usersCollection = getUsersCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)

In [ ]:
def dataGenerator():
    for interMin, interMax in [(300, 10000000), (30, 299), (10, 29), (3, 9), (2, 2)]:
        for _id, row in newsCollection.items():
            if dictContains(row, 'users') and len(row['users']) >= interMin and len(row['users']) <= interMax:
                yield {"id": row['url'], "content": {"title": "(" + str(len(row['users'])) + ") " + row['title'], "text": row['text']}}

In [ ]:
if False:
    def dataGenerator():
        for _id, row in newsCollection.items():
            yield {"id": row['url'], "content": {"title": row['title'], "text": row['text']}}

In [ ]:
labels = \
{
    "valid": {"title": "The document is valid", "type": LABEL_TYPE.checkbutton, "default": True},
}

In [ ]:
an = Annotator("valid-news", labels, useMongodb=True, user=hjuser, password=hjpass, host=hjhost, logger=logger)

In [ ]:
if not isNotebook:
    an.start(dataGenerator())

In [ ]:
if isNotebook:
    urlsToDelete = set()
    for url, values in an.data.items():
        if not values['valid']:
            urlsToDelete.add(url)
    bp(urlsToDelete, logger)
    log(len(urlsToDelete), logger)

In [ ]:
if isNotebook:
    for url in urlsToDelete:
        news = newsCollection.findOne({'url': url})
        users = news['users']
        for user in users:
            userData = usersCollection.findOne({'user_id': user})
            ns = userData['news']
            ts = userData['timestamps']
            newNs = []
            newTs = []
            for i in range(len(ns)):
                n = ns[i]
                t = ts[i]
                if n != url:
                    newNs.append(n)
                    newTs.append(t)
            assert len(newNs) < len(ns)
            assert len(newNs) == len(newTs)
            assert len(ns) == len(ts)
            if len(newNs) == 0:
                logError("The user " + str(user) + " has no more news.", logger)
            usersCollection.updateOne({'user_id': user}, {'$set': {'news': newNs, 'timestamps': newTs}})

In [ ]:
if isNotebook:
    for url in urlsToDelete:
        newsCollection.delete({'url': url})

In [ ]:
# Finally we delete user that has 0 news:
